In [ ]:
# import libraries
import pandas as pd
import numpy as np
import pickle

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, concatenate, Average
from tensorflow.keras.layers import Embedding
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers

from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D, LSTM, TimeDistributed, GRU, Bidirectional, Layer
from tensorflow.keras import backend as K

import tensorflow as tf
import matplotlib.pyplot as plt

## Training setup

In [ ]:
# load tokenizer
with open('notes_tokenizer_ps_find.pickle', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [ ]:
data = pd.read_csv("all_clinical_notes (Valid PS).csv")
data

In [ ]:
text_list = data["text_no_ps"].tolist()
text_len_list = [len(text) for text in text_list]

In [ ]:
train_data = data[data["split"] == "train"]
valid_data = data[data["split"] == "validation"]
test_data = data[data["split"] == "test"] 

In [ ]:
vocab_size = 10000
max_note_length = 2000

x_text_train = sequence.pad_sequences(tokenizer.texts_to_sequences([str(x) for x in train_data['text_no_ps']]), maxlen=max_note_length, padding='post')
x_text_valid = sequence.pad_sequences(tokenizer.texts_to_sequences([str(x) for x in valid_data['text_no_ps']]), maxlen=max_note_length, padding='post')
x_text_test = sequence.pad_sequences(tokenizer.texts_to_sequences([str(x) for x in test_data['text_no_ps']]), maxlen=max_note_length, padding='post')

In [ ]:
def get_simple_model(target):
    vocab_size = 10000
    embedding_dims = 256
    filters = 250
    kernel_size = 3
    epochs = 2
    hidden_dims = 128
    max_note_length=2000
    batch_size = 32

    # make model
    text_input = Input(shape=(max_note_length,), dtype='float32')

    text_embed = Embedding(vocab_size, embedding_dims, input_length=max_note_length, mask_zero=False)(text_input)
    
    cnn1 = Conv1D(filters=500, kernel_size=kernel_size, strides=1, padding='valid')(text_embed)
    x = GlobalMaxPooling1D()(cnn1)

    hidden = Dense(hidden_dims)(x)
    hidden = Activation('relu')(hidden)

    output = Dense(1, activation='linear')(hidden)

    model = Model(inputs=text_input, outputs=output)

    model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

## Training and validation

In [ ]:
# This is an unnecessary loop with only 1 element 'ps', kept here for convenience (old code)
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        filepath='ps_high'+'.h5',
        save_best_only=True,
        save_weights_only=True,
        monitor='val_loss'),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=1e-2,
        patience=10
    )
]

this_model = get_simple_model('ps_high')
this_model.fit(x_text_train, train_data['ps_high'].values,
         validation_data=(x_text_valid, valid_data['ps_high'].values),
         epochs=100,
         batch_size = 32, use_multiprocessing=True, callbacks=callbacks)

## Validation and test set evaluation

In [ ]:
this_model = get_simple_model('ps_high')
this_model.load_weights('ps_high'+'.h5')

In [ ]:
# helper functions to plot model metrics
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    import itertools
    import numpy as np


    from sklearn.metrics import confusion_matrix
  
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.grid(False)
    plt.show()

In [ ]:
# evaluate model auc
def eval_model(predicted, actual):
    from sklearn.metrics import roc_auc_score
    from sklearn.metrics import f1_score
    from sklearn.metrics import classification_report
    from sklearn.metrics import precision_recall_curve
    from sklearn.metrics import auc
    from sklearn.metrics import roc_curve

    print("AUC " + str(roc_auc_score(actual, predicted)))

    # calculate the fpr and tpr for all thresholds of the classification
    fpr, tpr, threshold = roc_curve(actual, predicted)
    roc_auc = auc(fpr, tpr)

    from sklearn.metrics import average_precision_score
    average_precision = average_precision_score(actual, predicted)

    print('Average precision score: {0:0.2f}'.format(
        average_precision))


    # method I: plt
    import matplotlib.pyplot as plt
    plt.title('Receiver Operating Characteristic: ' )
    plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
    plt.legend(loc = 'lower right')
    plt.plot([0, 1], [0, 1],'r--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('True Positive Rate')
    plt.xlabel('False Positive Rate')
    plt.show()



    import matplotlib.pyplot as plt
    #from sklearn.utils.fixes import signature

    precision, recall, thresholds = precision_recall_curve(actual, predicted)
    
    outcome_counts = np.unique(actual, return_counts=True)[1]
    prob_outcome = outcome_counts[1] / (outcome_counts[0] + outcome_counts[1])
    print('Outcome probability:')
    print(prob_outcome)
    
    plt.plot(recall, precision, color='b')
    plt.plot([0,1],[prob_outcome,prob_outcome], 'r--')
    plt.step(recall, precision, color='b', alpha=0.2,
             where='post')
    plt.fill_between(recall, precision, alpha=0.2, color='b')

    plt.xlabel('Recall (Sensitivity)')
    plt.ylabel('Precision (PPV)')
    plt.ylim([0.0, 1.05])
    plt.xlim([0.0, 1.0])
    plt.title('2-class Precision-Recall curve: AP={0:0.2f}'.format(
            average_precision))
    plt.show()


    # best F1
    F1 = 2*((precision*recall)/(precision+recall))
    print("Best F1 ")
    print(max(F1))
    
    # threshold for best F1
    bestF1_thresh = thresholds[np.argmax(F1)]
    print("Threshold for best F1:")
    print(bestF1_thresh)
    pred_outcome_best_f1_thresh = np.where(predicted >= bestF1_thresh,1,0)
    print(np.unique(pred_outcome_best_f1_thresh, return_counts=True))
    pred_outcome_00_thresh = np.where(predicted >= 0.0,1,0)
    
    # # predictions
    
    # # confusion matrix
    print("Confusion matrix at best F1 thresh:")
    from sklearn.metrics import confusion_matrix
    cnf_matrix = confusion_matrix(actual, pred_outcome_best_f1_thresh)
    np.set_printoptions(precision=2)
    # Plot non-normalized confusion matrix
    plt.figure()
    plot_confusion_matrix(cnf_matrix, classes=['No','Yes'],
                        title='Confusion matrix, without normalization')
    print("Metrics at best F1 thresh (specificity is recall for negative class):")
    from sklearn.metrics import classification_report
    print(classification_report(actual, pred_outcome_best_f1_thresh, target_names=['No','Yes']))


    print("Confusion matrix at 0.0 thresh:")
    from sklearn.metrics import confusion_matrix
    cnf_matrix = confusion_matrix(actual, pred_outcome_00_thresh)
    np.set_printoptions(precision=2)
    # Plot non-normalized confusion matrix
    plt.figure()
    plot_confusion_matrix(cnf_matrix, classes=['No','Yes'],
                        title='Confusion matrix, without normalization')
    print("Metrics at 0.0 thresh thresh (specificity is recall for negative class):")
    print(classification_report(actual, pred_outcome_00_thresh, target_names=['No','Yes']))

    # # plot threshold vs ppv curve
    plt.plot(thresholds, precision[0:len(precision)-1], color='b')

    plt.xlabel('Threshold probability')
    plt.ylabel('Precision (PPV)')
    plt.ylim([0.0, 1.0])
    plt.xlim([0.0, 1.0])
    plt.title('Threshold vs precision')
    plt.show()

    # histogram
    plt.hist(predicted)
    plt.title("Histogram")
    plt.xlabel("Predicted probability" )
    plt.ylabel("Frequency")
    plt.show()

In [ ]:
val_logits_list = this_model.predict(x_text_valid)
eval_model(val_logits_list, valid_data['high_ps'].values)

In [ ]:
test_logits_list = this_model.predict(x_text_test)
eval_model(test_logits_list, test_data['high_ps'].values)